# Credit Card Fraud

Credit card fraud is a problem that finance companies which offer consumers credit cards would like to eliminate. Since there are around 1 billion credit card transactions which take place every day, manually flagging and cancelling fraudulent transactions is entirely unfeasible. With machine learning, we can train a computer to distinguish fraudulent transactions from real ones using previous examples. This protects the company from liability and loss and also reduces operating costs.

The following dataset is from Kaggle (https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud). The dataset contains transactions made with credit cards by European cardholders in September 2013. The data has gone through dimensionality reduction via PCA, which anonymizes the data and also makes it easier for a machine learning model to learn from. The only features which remain untransformed are the 'Time' and 'Amount' features. The 'Time' column contains the seconds elapsed between each transaction and the first transaction in the dataset. The 'Amount' feature is the transaction amount. Finally, the 'Class' feature is the target variable, with '1' being a fraudulent transaction and 0 otherwise.

The dataset author recommends: 
> Given the class imbalance ratio, we recommend measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). Confusion matrix accuracy is not meaningful for unbalanced classification.